# Setting

## Import Library

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Date
from calendar import monthrange
from datetime import date, datetime

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

## Set Date Functions
모든 데이터에 대한 주차별 데이터를 사용할 예정이므로  
날짜 정보를 주차 컬럼으로 변경하는 함수 정의

In [2]:
from datetime import date, timedelta


def set_week(df, date):
    '''
    df : datetime 형식의 컬럼을 가지고 있는 dataframe
    date : df에서 datetime 형식을 가진 컬럼명
    return : date의 연도 컬럼과 주차 컬럼을 추가한 dataframe
    '''
    df[date] = pd.to_datetime(df[date])
    df[date] = df[date].dt.date
    df['year'] = df.apply(func=lambda x: x[date].isocalendar()[0], axis=1)
    df['week'] = df.apply(func=lambda x: x[date].isocalendar()[1], axis=1)
    df.drop(date, axis=1, inplace=True)
    

def check_week(df):
    '''
    df에 date가 전부 있는지 확인
    '''
    cnt = 0
    sdate = date(2015, 12, 28)   # start date
    edate = date(2019, 12, 30)   # end date
    delta = edate - sdate       # as timedelta
    mem = set()
    
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        year, week = day.isocalendar()[0], day.isocalendar()[1]
        if year * 100 + week in mem:
            continue
        mem.add(year * 100 + week)
        if df[(df['year'] == year) & (df['week'] == week)].empty:
            print((year, week), end="")
            cnt += 1
    if cnt > 0:
        print()
    print("missing", cnt, "values")    

## Set Data Root Directory

In [3]:
root = os.path.join(os.getcwd(), 'DATA')

# Weather

## Import Data

In [4]:
df_weather_code = pd.read_csv(os.path.join(root, 'raw_weather_code.csv'), header=0, index_col=0)

In [5]:
weather_list = [pd.read_csv(os.path.join(root, 'raw_weather_20151228_20161227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20161228_20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20171228_20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20181228_20191227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20191228_20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20201228_20210818.csv'), encoding='euc-kr')]


## Preprocess
- '지점'에 따른 나라명 컬럼 (country) 추가
- 각 나라, 일자 별로 평균 강수량, 풍속, 기온 계산

In [6]:
# 지점에 따라 나라명 추가
def set_country(row):
    data = df_weather_code[df_weather_code['지점'] == row['지점']]
    if data.empty:
        return ""
    return data.iloc[0]['국가명']


def set_waterfront(row):
    data = df_weather_code[df_weather_code['지점'] == row['지점']]
    if data.empty or data.iloc[0]['해안가여부'] != 1:
        return False
    return True
    

def preprocess_weather(df):
    df = df[(df['강수량'] >= 0) & (df['풍속'] >= 0)] # 이상치/결측치 제거
    set_week(df, '일시')  # 날짜 정보 처리
    
    # 1차 평균
    columns = ['year', 'week', '지점']
    df['rain'] = df[columns + ['강수량']].groupby(columns).transform('mean')
    df['wind'] = df[columns + ['풍속']].groupby(columns).transform('mean')
    df['temperature'] = df[columns + ['기온']].groupby(columns).transform('mean')
    
    # 나라명 추가
    df['CTRY_1'] = ""
    for i, row in df.iterrows():
        df.at[i, 'CTRY_1'] = set_country(row)
    
    # 해안가 여부 추가
    df['is_waterfront'] = False
    for i, row in df.iterrows():
        df.at[i, 'is_waterfront'] = set_waterfront(row)
        
#     df = df[df['is_waterfront']]  # 해안가가 아닌 데이터 제외
    
    # 2차 평균
    columns = ['year', 'week', 'CTRY_1']
    df['rain'] = df[columns + ['강수량']].groupby(columns).transform('mean')
    df['wind'] = df[columns + ['풍속']].groupby(columns).transform('mean')
    df['temperature'] = df[columns + ['기온']].groupby(columns).transform('mean')

    # 컬럼/행 정리
    df.drop(columns=['지점명', '지점', '강수량', '풍속', '기온', 'is_waterfront'], inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [7]:
for i in range(len(weather_list)):
    weather_list[i] = preprocess_weather(weather_list[i])

In [8]:
df_weather = pd.concat(weather_list)

In [9]:
df_weather.describe()

,year,week,rain,wind,temperature
count,1679.000000,1679.000000,1679.000000,1679.000000,1679.000000
mean,2018.232877,25.856462,5.981158,3.642640,14.906772
std,1.607481,15.124088,32.371218,3.200572,10.536941
min,2015.000000,1.000000,0.000000,0.000000,-21.600000
25%,2017.000000,13.000000,1.489908,1.869608,8.266627
50%,2018.000000,25.000000,2.750000,2.452555,15.382857
75%,2020.000000,39.000000,6.504304,5.111601,25.430698
max,2021.000000,53.000000,915.000000,60.000000,30.100000


## Check

결측값 확인

In [10]:
for country in df_weather['CTRY_1'].unique():
    if country == "":
        continue
    print()
    print(country, "총", len(df_weather[df_weather['CTRY_1'] == country]), "개")
    check_week(df_weather[df_weather['CTRY_1'] == country])


노르웨이 총 236 개
missing 0 values

태국 총 297 개
(2016, 11)
missing 1 values

베트남 총 298 개
(2016, 6)
missing 1 values

중국 총 299 개
missing 0 values

페루 총 7 개
(2015, 53)(2016, 1)(2016, 2)(2016, 3)(2016, 4)(2016, 5)(2016, 6)(2016, 7)(2016, 8)(2016, 9)(2016, 10)(2016, 11)(2016, 12)(2016, 13)(2016, 14)(2016, 15)(2016, 16)(2016, 17)(2016, 18)(2016, 19)(2016, 20)(2016, 21)(2016, 22)(2016, 23)(2016, 24)(2016, 25)(2016, 26)(2016, 27)(2016, 28)(2016, 29)(2016, 30)(2016, 31)(2016, 32)(2016, 33)(2016, 34)(2016, 35)(2016, 36)(2016, 37)(2016, 38)(2016, 39)(2016, 40)(2016, 41)(2016, 42)(2016, 43)(2016, 44)(2016, 45)(2016, 46)(2016, 47)(2016, 48)(2016, 49)(2016, 51)(2016, 52)(2017, 1)(2017, 2)(2017, 3)(2017, 4)(2017, 5)(2017, 6)(2017, 7)(2017, 8)(2017, 9)(2017, 10)(2017, 12)(2017, 13)(2017, 14)(2017, 15)(2017, 16)(2017, 17)(2017, 18)(2017, 19)(2017, 20)(2017, 21)(2017, 22)(2017, 23)(2017, 24)(2017, 25)(2017, 26)(2017, 27)(2017, 28)(2017, 29)(2017, 30)(2017, 31)(2017, 32)(2017, 33)(2017, 34)(2017, 35)(2017, 3

## Filling Missing Values

- 노르웨이, 중국은 결측치 없음
- 태국, 베트남, 칠레는 결측치 1개
> 전/차주 데이터 평균으로 채워넣기
- 페루는 결측치 299개
> 사용 불가로 판단

In [11]:
# 페루 데이터 제거

df_weather = df_weather[(df_weather['CTRY_1'] == '태국') | (df_weather['CTRY_1'] == '중국') | (df_weather['CTRY_1'] == '베트남') | (df_weather['CTRY_1'] == '노르웨이') | (df_weather['CTRY_1'] == '칠레')]

In [12]:
def get_avg(year, week, country, value):
  return (df_weather[(df_weather['year'] == year) & (df_weather['week'] == week + 1) & (df_weather['CTRY_1'] == country)].iloc[0][value] 
          + df_weather[(df_weather['year'] == year) & (df_weather['week'] == week - 1) & (df_weather['CTRY_1'] == country)].iloc[0][value]) / 2

In [13]:
df_weather = df_weather.append({'year': 2016,
                                'week': 11,
                                'CTRY_1': '태국',
                                'rain': get_avg(2016, 11, '태국', 'rain'),
                                'wind': get_avg(2016, 11, '태국', 'wind'),
                                'temperature': get_avg(2016, 11, '태국', 'temperature')},
                               ignore_index=True)
df_weather = df_weather.append({'year': 2016,
                                'week': 6,
                                'CTRY_1': '베트남',
                                'rain': get_avg(2016, 6, '베트남', 'rain'),
                                'wind': get_avg(2016, 6, '베트남', 'wind'),
                                'temperature': get_avg(2016, 6, '베트남', 'temperature')},
                               ignore_index=True)
df_weather = df_weather.append({'year': 2016,
                                'week': 5,
                                'CTRY_1': '칠레',
                                'rain': get_avg(2016, 5, '칠레', 'rain'),
                                'wind': get_avg(2016, 5, '칠레', 'wind'),
                                'temperature': get_avg(2016, 5, '칠레', 'temperature')},
                               ignore_index=True)

for country in ["태국", "베트남", "칠레"]:
    print()
    print(country, "총", len(df_weather[df_weather['CTRY_1'] == country]), "개")
    check_week(df_weather[df_weather['CTRY_1'] == country])


태국 총 298 개
missing 0 values

베트남 총 299 개
missing 0 values

칠레 총 298 개
missing 0 values


# Oil

## Import Data

In [14]:
df_oil = pd.read_csv(os.path.join(root, 'raw_oil.csv'), usecols=["날짜", "종가"])
df_oil_dubai = pd.read_csv(os.path.join(root, 'raw_oil_dubai.csv'), usecols=["날짜", "종가"])
df_oil_brent = pd.read_csv(os.path.join(root, 'raw_oil_brent.csv'), usecols=["날짜", "종가"])

## Preprocess

In [15]:
from datetime import datetime
def preprocess_oil(df):
    df['date'] = pd.to_datetime(df['날짜'].str[:4] + df['날짜'].str[6:8] + df['날짜'].str[10:12])
    set_week(df, 'date')  # 날짜 데이터 정리
    df['oil'] = df[['year', 'week', '종가']].groupby(['year', 'week']).transform('mean')
    df.drop(columns = ['날짜', '종가'], inplace=True, axis=1)
    df.drop_duplicates(inplace=True)
    return df

In [16]:
df_oil = preprocess_oil(df_oil)
df_oil_dubai = preprocess_oil(df_oil_dubai)
df_oil_brent = preprocess_oil(df_oil_brent)

## Check

In [17]:
for i in range(1, 53):
    if i not in (list(df_oil_brent[df_oil_brent['year'] == 2019].sort_values(by=['year', 'week'])['week'])):
        print(i)

In [18]:
check_week(df_oil)
check_week(df_oil_dubai)
check_week(df_oil_brent)

missing 0 values
missing 0 values
missing 0 values


oil과 비교하였을 때 dubai의 경우 28개, brent의 경우 3개의 데이터가 적다  
따라서 df_oil 데이터를 사용하도록 한다

---

# Korea Weather

## Import Data

In [19]:
df_weather_kr = pd.read_csv(os.path.join(root, 'raw_weather_korea.csv'),
                            encoding='euc-kr',
                            usecols=["일시", "평균 풍속(m/s)", "평균 기온(°C)", "평균 수온(°C)"])

## Preprocess Date

In [20]:
df_weather_kr['date'] = pd.to_datetime(df_weather_kr['일시'].str[:4] + df_weather_kr['일시'].str[5:7] + df_weather_kr['일시'].str[8:])
set_week(df_weather_kr, 'date')

In [21]:
df_weather_kr.describe()

,평균 풍속(m/s),평균 기온(°C),평균 수온(°C),year,week
count,35181.000000,35107.000000,35300.000000,35558.000000,35558.000000
mean,5.732987,14.845689,16.884377,2018.369143,25.531554
std,2.705727,7.573504,5.945493,1.627939,15.146644
min,0.000000,-11.100000,1.100000,2015.000000,1.000000
25%,3.600000,8.800000,13.000000,2017.000000,13.000000
50%,5.400000,15.100000,16.600000,2018.000000,24.000000
75%,7.500000,21.000000,21.400000,2020.000000,39.000000
max,19.600000,34.600000,31.500000,2021.000000,53.000000


## Group
일자 별로 평균 구하기

In [22]:
df_weather_kr['wind_kr'] = df_weather_kr[['year', 'week', '평균 풍속(m/s)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['temperature_kr'] = df_weather_kr[['year', 'week', '평균 기온(°C)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['water_temp_kr'] = df_weather_kr[['year', 'week', '평균 수온(°C)']].groupby(['year', 'week']).transform('mean')

## Drop Column

In [23]:
drop = ['일시', '평균 풍속(m/s)', '평균 기온(°C)', '평균 수온(°C)']

In [24]:
df_weather_kr.drop(columns=drop, inplace=True, axis=1)
df_weather_kr.drop_duplicates(inplace=True)

## Check

In [25]:
check_week(df_weather_kr)

missing 0 values


In [26]:
df_weather_kr.describe()

,year,week,wind_kr,temperature_kr,water_temp_kr
count,288.000000,288.000000,288.000000,288.000000,288.000000
mean,2018.267361,25.510417,5.723881,14.920397,16.953522
std,1.612682,15.112121,1.340964,7.177077,5.321755
min,2015.000000,1.000000,2.706723,-0.529915,8.389899
25%,2017.000000,12.750000,4.763866,8.648878,12.088782
50%,2018.000000,24.500000,5.668908,14.792355,16.279115
75%,2020.000000,38.250000,6.656410,21.312062,21.471801
max,2021.000000,53.000000,9.819643,28.610924,28.495798


---

# CPI

## Import Data

In [27]:
df_cpi_total = pd.read_csv(os.path.join(root, 'raw_cpi_total.csv'), encoding='cp949')  # 총 소비자물가지수
df_cpi_fish =  pd.read_csv(os.path.join(root, 'raw_cpi_fish.csv'), encoding='cp949')  # 수산물 소비자물가지수

## Drop Columns/Rows

In [28]:
df_cpi_total = df_cpi_total[df_cpi_total['지출목적별'] == "0 총지수"].drop(columns=['시도별', '지출목적별'], axis=1)
df_cpi_fish = df_cpi_fish[df_cpi_fish['지출목적별'] == "어류 및 수산"].drop(columns=['Unnamed: 0', '지출목적별'], axis=1)

## Pivoting

In [29]:
df_cpi_total = df_cpi_total.transpose().reset_index()
df_cpi_fish = df_cpi_fish.transpose().reset_index()

## Rename

In [30]:
df_cpi_total.rename(columns={0: 'cpi_total'}, inplace=True)
df_cpi_fish.rename(columns={0: 'cpi_fish'}, inplace=True)
print(df_cpi_total.columns, df_cpi_fish.columns)

Index(['index', 'cpi_total'], dtype='object') Index(['index', 'cpi_fish'], dtype='object')


## Preprocess Date

In [31]:
df_cpi_total['year'] = df_cpi_total['index'].str[:4].astype('int')
df_cpi_total['month'] = df_cpi_total['index'].str[6:8].astype('int')
df_cpi_fish['year'] = df_cpi_fish['index'].str[:4].astype('int')
df_cpi_fish['month'] = df_cpi_fish['index'].str[6:8].astype('int')

In [32]:
df_cpi_fish

,index,cpi_fish,year,month
0,2015. 12 월,101.26,2015,12
1,2016. 01 월,100.76,2016,1
2,2016. 02 월,102.30,2016,2
3,2016. 03 월,102.23,2016,3
4,2016. 04 월,102.73,2016,4
...,...,...,...,...
63,2021. 03 월,121.67,2021,3
64,2021. 04 월,121.79,2021,4
65,2021. 05 월,122.01,2021,5
66,2021. 06 월,121.50,2021,6


In [33]:
df_cpi_total.drop(columns=['index'], inplace=True)
df_cpi_fish.drop(columns=['index'], inplace=True)

In [34]:
# Merge df
df_cpi = pd.merge(left=df_cpi_total, right=df_cpi_fish, how='left', on=['year', 'month'])

In [35]:
# create days by duplicating data
new_df = pd.DataFrame(columns=list(df_cpi.columns) + ['day'])
for i, row in df_cpi.iterrows():
    data = df_cpi.iloc[i]
    year, month, cpi_total, cpi_fish = data['year'].astype('int'), data['month'].astype('int'), data['cpi_total'], data['cpi_fish']
    for day in range(1, monthrange(year, month)[1]+1):
        if datetime(year, month, day).weekday() == 0:
            new_df = new_df.append({'year': str(year),
                                    'month': str(month),
                                    'day': str(day),
                                    'cpi_total': cpi_total,
                                    'cpi_fish': cpi_fish},
                                   ignore_index=True)
df_cpi = new_df

In [36]:
df_cpi['date'] = df_cpi['year'] + " " + df_cpi['month'] + " " + df_cpi['day']

In [37]:
set_week(df_cpi, 'date')

## 정리
- drop unused columns
- drop duplicates

In [38]:
df_cpi.drop(columns=['month', 'day'], inplace=True)
df_cpi.drop_duplicates(inplace=True)

## Check

In [39]:
check_week(df_cpi)

missing 0 values


---

# Exchange Rate

In [40]:
exchange_list = [pd.read_csv(os.path.join(root, 'raw_exchange_chile.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_china.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_norway.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_peru.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_thai.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_vietnam.csv'), usecols=["날짜", "종가"])]

In [41]:
ctry_name = ['칠레', '중국', '노르웨이', '페루', '태국', '베트남']
for i in range(len(exchange_list)):
    set_week(exchange_list[i], '날짜')
    check_week(exchange_list[i])
    exchange_list[i]['CTRY_2'] = ctry_name[i]

missing 0 values
missing 0 values
missing 0 values
missing 0 values
missing 0 values
missing 0 values


In [42]:
df_exchange = pd.concat(exchange_list)

## Rename

In [43]:
df_exchange.rename(columns={"종가": 'exchange'}, inplace=True)

## Save Files

In [44]:
def save_file(df, file_name):
    df.to_csv(os.path.join(root, file_name), encoding='utf-8', index=False)

In [45]:
save_file(df_weather, 'preprocessed_weather.csv')  # 세계 날씨
save_file(df_oil, 'preprocessed_oil.csv')  # 원유 종가
save_file(df_weather_kr, 'preprocessed_weather_korea.csv')  # 한국 날씨
save_file(df_cpi, 'preprocessed_cpi.csv')  # 소비자물가지수
save_file(df_exchange, 'preprocessed_exchange.csv')  # 환율

# 끝